In [40]:
# Here define data directory and path to metadata csv file
from operator import itemgetter
import numpy as np
tmp=['A','B','C','A','C']
e,indeces=np.unique(tmp, return_index=True)
h=[]
h.append('/ab/test/this')
h.append('/cb/asdf/this')
h.append('/bf/asdd/this')
h.append('/gh/asdf/this')

ind=[0,3]
print ind
print itemgetter(*ind)(h)

j=['a','b','c','d']
j.index('dd')

[0, 3]
('/ab/test/this', '/gh/asdf/this')


ValueError: 'dd' is not in list

In [4]:
import pandas as pd
import numpy as np
import glob, os
from operator import itemgetter

metadata_sheet='Table'
data_path='/nfs/zorba/DOWNLOADED/Enhanced_NKI/data/nki/dicom/triotim/'
# Below will capture all the structural and functional dicom folders (some are duplicates)
#T1_glob_data = '*/*/A*/*_V*/MPRAGE*/co*nii.gz' # Grabs all 
#T1_glob_data = '*/*/A*/*_V{2,A,_ALG}/MPRAGE*/co*nii.gz' # Grabs all baseline
T1_glob_data = '*/*/A*/*_V[2,A,_ALG]/MPRAGE*/co*nii.gz' # Grabs all baseline, later can add the repeat session
func_glob_data = '*/*/A*/*_V[2,A,_ALG]/REST*/*.nii.gz' # Come back and incorporate
os.chdir(data_path)
T1_scans=sorted(glob.glob(T1_glob_data)) # Focus on T1 scans for now
func_scans=sorted(glob.glob(func_glob_data))
print "length of structural scans is: " + str(len(T1_scans))
print "length of functional scans is: " + str(len(func_scans))
# Now get only unique ids for structural scans
tmp=[]
for s in T1_scans:
    tmp.append(s.split('/')[2])
    
T1_subject_ids,indeces=np.unique(tmp, return_index=True)
# Now T1_scans list is only filled with unique values
T1_scans=itemgetter(*indeces)(T1_scans)
print "length of unique structural scans is: " + str(len(T1_scans))

# Now get only unique ids for functional lores resting scans
tmp=[]
for s in func_scans:
    tmp.append(s.split('/')[2])
    
func_subject_ids,indeces=np.unique(tmp, return_index=True)
# Now T1_scans list is only filled with unique values
func_scans=itemgetter(*indeces)(func_scans)
print "length of unique functional scans is: " + str(len(func_scans))

dict_scan = {}
### Read and Parse XSL
# metadata_file='/Users/haiyuancao/GitHub/xnat_import/scan-new/8100_IRI_20161115_2.csv'

metadata_file_2 = '/nfs/zorba/Dropbox/NKI/8100_Demos_20170117.csv'
df = pd.read_csv(metadata_file_2)
#df['Anoymized ID'] = df['Anoymized ID'].apply(lambda x: unicode(x))
#id_list = df['Anonymized ID'].values.tolist()
df_2 = df.set_index('Anonymized ID')
id_list = df_2.index.tolist()
subject_dict = {}
count=0

for i in range(len(T1_subject_ids)):
    item=T1_subject_ids[i] # Item is the subject ID "i.e. A00053278"
    T1_path=T1_scans[i]
    #print (item)
    #print file_path
    if item in id_list:
        # Here go ahead and grab the subject's lores REST func_path too
        try:
            func_index = func_subject_ids.tolist().index(item)
            func_path  = func_scans[func_index]
        except: 
            func_path = []
        #print id_list.index(item)
        count += 1
        df_index =id_list.index(item)
        age = df_2.iloc[df_index]['DEM_001']
        sex = df_2.iloc[df_index]['DEM_002']
        
        #print (age, sex)
        subject_dict[item] = {'age': "", "sex": "", "T1_path":"", "func_path":""}
        subject_dict[item]['age'] = age
        subject_dict[item]['sex'] = sex
        subject_dict[item]['T1_path'] = T1_path
        subject_dict[item]['func_path'] = func_path
       
print (count, len(T1_subject_ids))

for key, value in subject_dict.items()[0:5]:
    print key  
    print value["T1_path"]
        

length of structural scans is: 942
length of functional scans is: 903
length of unique structural scans is: 938
length of unique functional scans is: 891
(871, 938)
A00052183
mmilham/long_child_483009/A00052183/418858611_V2/MPRAGE_SIEMENS_DEFACED_0003/co18991230_000000A00052183s003a1001.nii.gz
A00040248
mmilham/discoverysci_30001/A00040248/395755775_V2/MPRAGE_SIEMENS_DEFACED_0011/co18991230_000000A00040248s011a1001.nii.gz
A00064323
mmilham/discoverysci_30001/A00064323/466670035_VA/MPRAGE_SIEMENS_DEFACED_0003/co18991230_000000A00064323s003a1001.nii.gz
A00044405
mmilham/discoverysci_30001/A00044405/411163737_V2/MPRAGE_SIEMENS_DEFACED_0003/co18991230_000000unknowns003a1001.nii.gz
A00053744
mmilham/long_child_483009/A00053744/421546114_V2/MPRAGE_SIEMENS_DEFACED_0003/co18991230_000000A00053744s003a1001.nii.gz


In [11]:
import pyxnat
import os
delete_previous_scans = 0
insert_scans = 1

# Set project name
project_name="NKI"

# connect to XNAT instance
xnat=pyxnat.Interface(config='/home/spantazatos/xnat_sp.cfg')

# Create the project in XNAT
# Create project object
project=xnat.select.project(project_name)

# First check if it already exists
if not project.exists():
    project.insert()
    
for key, value in subject_dict.items():
    print key
    print value["T1_path"]
    
    # Insert subject if not already there
    print ("Now insert subject with key: " + key)
    subject = project.subject(key)
    if not subject.exists():
        subject.insert()
    
    # Since this is baseline just do MR1 for now
    session_label = "%s_%s_MR1" % (project_name,key)
    print session_label
    
    experiment=subject.experiment(session_label) #.create(experiments='xnat:mrSessionData')
   
    # Create experiment object, insert if it doesn't already exst
    if not experiment.exists():
        experiment.create() # By default it will make an xnat:mrSessionData experiment which is what we need
    
    # Now insert subject-level attributes (i.e. SEX) and experiment-level attributes (i.e. AGE)
    user_age = int(value['age'])
    if value['sex'] == 1:
        user_sex = 'Male'
    else:
        user_sex = 'Female'
    
#     sex = 'Male'
#     weight = str(100)
#     height = str(100)
    
    subject.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/gender', user_sex)
    subject.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/age', str(user_age))
    #subject.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/height', height)
    
    model ='SIEMENS MAGNETOM TrioTim syngo MR B17'
    site ='Hammersmith Hospital'
    fs ='3T'
    
    experiment.attrs.set('xnat:imageSessionData/scanner',model)
    experiment.attrs.set('xnat:imageSessionData/fieldStrength',fs)

    # Below for inserting scans
    if insert_scans==1:
        scan_files=[]
        scan_files.append(value['T1_path'])
        if value['func_path'] != []:
            scan_files.append(value['func_path'])
    
        # Delete old scans if desired
        if delete_previous_scans==1:
            allScans=experiment.scans()
            for scan in allScans:
                print "deleting scans"
                scan.delete()

        for ii in range(len(scan_files)):
            if ii==0:
                scan_type='T1'
                scan_name='T1_01'
                slice_size='8 1200'
            elif ii==1:
                scan_type='func_rest'
                scan_name='func_rest_02'
                slice_size='3 267'

            img_reorient='anonymized_reorient.nii.gz'
            #cmd_reorient='fslreorient2std ' + value['T1_path'] + ' ' + img_reorient
            cmd_reorient='cp ' + scan_files[ii] + ' ' + img_reorient # Don't reorient since already including dcm2nii orientation
            os.system(cmd_reorient)

            # Now insert the reoriented image
            # experiment.scan('1').resource('NIFTI').file(os.path.basename(scans_filtered[sub])).insert(
            experiment.scan(scan_name).resource('NIFTI').file(os.path.basename(img_reorient)).insert(
            img_reorient,
            content=scan_type,
            overwrite=True,
            type=scan_type,  # May need to change this to SPGR, MPRAGE, FLAIR etc?
            format='NIFTI')

            # Create ORIGINAL and SNAPSHOT to view the images in the web browser
            img_png='tmp.png'
            img_gif='tmp.gif'
            img_thumb='tmp_thumb.gif'
            cmd_create_slices='slicer ' + img_reorient + ' -S ' + slice_size + ' ' + img_png
            cmd_convert_gif='convert ' + img_png + ' ' + img_gif
            cmd_convert_thumb='convert ' + img_png + ' -resize 240x240 ' + img_thumb

            os.system(cmd_create_slices)
            os.system(cmd_convert_gif)
            os.system(cmd_convert_thumb)

            experiment.scan(scan_name).resource('SNAPSHOTS').file(os.path.basename(img_gif)).insert(
            img_gif,
            content='ORIGINAL',
            overwrite=True,
            format='GIF')

            experiment.scan(scan_name).resource('SNAPSHOTS').file(os.path.basename(img_thumb)).insert(
            img_thumb,
            content='THUMBNAIL',
            overwrite=True,
            format='GIF')
                                                    

#xnat.select.project('demoProj_1').accessibility()
#xnat.select.project('demoProj_1').set_accessibility(accessibility='private')

#db.select.project('IXI_TEST').create()
print "ok"
    

A00052183
mmilham/long_child_483009/A00052183/418858611_V2/MPRAGE_SIEMENS_DEFACED_0003/co18991230_000000A00052183s003a1001.nii.gz
Now insert subject with key: A00052183
NKI_A00052183_MR1
A00040248
mmilham/discoverysci_30001/A00040248/395755775_V2/MPRAGE_SIEMENS_DEFACED_0011/co18991230_000000A00040248s011a1001.nii.gz
Now insert subject with key: A00040248
NKI_A00040248_MR1
A00064323
mmilham/discoverysci_30001/A00064323/466670035_VA/MPRAGE_SIEMENS_DEFACED_0003/co18991230_000000A00064323s003a1001.nii.gz
Now insert subject with key: A00064323
NKI_A00064323_MR1
A00044405
mmilham/discoverysci_30001/A00044405/411163737_V2/MPRAGE_SIEMENS_DEFACED_0003/co18991230_000000unknowns003a1001.nii.gz
Now insert subject with key: A00044405
NKI_A00044405_MR1
A00053744
mmilham/long_child_483009/A00053744/421546114_V2/MPRAGE_SIEMENS_DEFACED_0003/co18991230_000000A00053744s003a1001.nii.gz
Now insert subject with key: A00053744
NKI_A00053744_MR1
A00043758
mmilham/discoverysci_30001/A00043758/408377626_V2/MPRA